In [1]:
import model
import data
import tensorflow as tf
import numpy as np
import random

To avoid verbose warning messages... 

In [2]:
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

Load MNIST dataset via a Dataset object:

In [3]:
dataset = data.Dataset(batch_size=128)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
55000


## Main Model Training

- ### Teacher Model

Training parameters:

In [4]:
learning_rate=0.001
num_steps = 500
batch_size = 128

Model parameters:

In [5]:
temperature = 1.0
dropout = 0.75

Model definition:

In [6]:
teacher_model = model.BigModel(num_steps=num_steps, 
                               batch_size=batch_size,
                               learning_rate=learning_rate,
                               temperature=temperature,
                               dropoutprob=dropout,
                               model_type="teacher");

Training:

At each step, the validation accuracy is computed and, if maximal, a model checkpoint is created. This is, in a way, analogous to **early stopping**.

In [7]:
teacher_model.start_session()
teacher_model.train(dataset)

Starting Training
Model Checkpointed to checkpoint\bigmodel.ckpt 
Step 1, Validation Loss= 137457.2031, Validation Accuracy= 0.106
Model Checkpointed to checkpoint\bigmodel.ckpt 
Step 100, Validation Loss= 2454.9197, Validation Accuracy= 0.861
Model Checkpointed to checkpoint\bigmodel.ckpt 
Step 200, Validation Loss= 1520.0682, Validation Accuracy= 0.908
Model Checkpointed to checkpoint\bigmodel.ckpt 
Step 300, Validation Loss= 1123.4905, Validation Accuracy= 0.930
Model Checkpointed to checkpoint\bigmodel.ckpt 
Step 400, Validation Loss= 854.3755, Validation Accuracy= 0.940
Model Checkpointed to checkpoint\bigmodel.ckpt 
Step 500, Validation Loss= 671.8576, Validation Accuracy= 0.950
Model Checkpointed to checkpoint\bigmodel.ckpt 
Optimization Finished!


Test the **teacher model** (compute its accuracy againts the testing dataset) on the best model based on the validation set, this is, the *checkpointed model*. 

In [10]:
checkpoint_dir = "checkpoint"
# Load the best model from created checkpoint
teacher_model.load_model_from_file(checkpoint_dir)
# Test the model against the testing set
teacher_model.run_inference(dataset)

Reading model parameters from checkpoint\bigmodel.ckpt
Testing Accuracy: 0.949137


In [11]:
# Close current tf session
teacher_model.close_session()

- ### Simple Student Model
Simple, as in, trained with the same data as the teacher model. 

- ### Distilled Student Model
Training data consists of the **logits** from the Teacher Model predictions of the standard training set.

## Experiments

- ### 1. Learn from Probabilities
Take a class out from the training set for the Distilled Model, and later test the accuracy for that class. 